In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import sklearn
#from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import multiprocessing
import scipy
from sklearn.model_selection import KFold

# 读取数据

In [2]:
x_data = pd.read_excel("All features.xlsx")
x_data

c1nc[nH]n1  c1c[nH]cn1  c1cc[nH]c1  c1ccncc1  c1ccc2[nH]ccc2c1  \
0            0           0           0         0                 0   
1            0           0           0         0                 0   
2            0           0           0         0                 0   
3            0           0           0         0                 0   
4            0           0           0         0                 0   
..         ...         ...         ...       ...               ...   
77           0           0           0         0                 0   
78           0           0           0         0                 0   
79           0           0           0         0                 0   
80           0           0           0         0                 0   
81           0           0           0         0                 0   

    c1ccc2scnc2c1  c1ccc2cc3ccccc3cc2c1  c1cscn1  c1ccsc1  c1cn[nH]c1  \
0               0                     0        0        1           0   
1               0                     0        0        1           0   
2               0                     0        0        0           0   
3               0                     0        0        0           0   
4               0                     0        0        0           0   
..            ...                   ...      ...      ...         ...   
77              0                     0        0        0           0   
78              0                     0        0        0           0   
79              0                     0        0        0           0   
80              0                     0        0        0           0   
81              0                     0        0        0           0   

    c1ccc2occcc2c1  c1cc2ccc(ccnccc3ccc(ccn1)[nH]3)[nH]2  c1ccccc1  \
0                0                                     0         0   
1                0                                     0         0   
2                0                                     0         0   
3                0                                     0         0   
4                0                                     0         0   
..             ...                                   ...       ...   
77               0                                     0         0   
78               0                                     0         0   
79               0                                     0         0   
80               0                                     0         0   
81               0                                     0         0   

    c1nnn[nH]1  c1ccc2sc[n+]c2c1  c1ccc2ncccc2c1  c1nncs1  Alkene  \
0            0                 0               0        0       0   
1            0                 0               0        0       0   
2            0                 0               0        0       0   
3            0                 0               0        0       0   
4            0                 0               0        0       2   
..         ...               ...             ...      ...     ...   
77           0                 0               0        0       0   
78           0                 0               0        0       0   
79           0                 0               0        0       0   
80           0                 0               0        0       0   
81           0                 0               0        0       0   

    Alkylchloride  Alkylfluoride  Alkylbromide  Trifluoromethyl  Alcohol  \
0               0              0             0                0        0   
1               0              0             0                0        0   
2               0              0             0                0        0   
3               0              0             0                0        0   
4               0              0             0                0        0   
..            ...            ...           ...              ...      ...   
77              0              0             0                0        0   
78      

In [3]:
y_data = pd.read_csv("Target.csv", usecols=[1])
y_data

FEaverage CO
0      14.760128
1      18.144924
2      20.750142
3      19.887920
4      21.001702
..           ...
77      6.013192
78      3.219462
79      2.026439
80     18.885723
81      8.596870

[82 rows x 1 columns]

In [4]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
x_data.columns = [regex.sub("_", col) if any(i in str(col) for i in set(('[', ']', '<'))) 
                        else col for col in x_data.columns.values]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=0)

In [5]:
scaler1 = preprocessing.MinMaxScaler().fit(x_train)  # 通过训练集获得归一化函数模型，使用 x_train 数据来计算缩放器需要的最小值和最大值
scaler2 = preprocessing.MinMaxScaler().fit(y_train)

x_train = scaler1.transform(x_train)#使用拟合好的缩放器对 x_train 数据进行缩放
x_test = scaler1.transform(x_test)

y_train = scaler2.transform(y_train)
y_test = scaler2.transform(y_test)

In [6]:
num_cores = multiprocessing.cpu_count()

In [8]:
kf = KFold(n_splits=10, random_state=0, shuffle=True) # 5折交叉验证
#mse它测量的是预测值与实际值之间差值的平方的平均值。均方误差越小，模型的预测性能越好
variate=list(range(x_train.shape[1]))   #变量索引列表
selected=[]   #变量索引
train_mse_list = []
val_mse_list = []

current_score,best_new_score=float('inf'),float('inf')  
#验证集目前的分数和最好分数初始值都为无穷大（因为mse越小越好）
    
# def my_custom_loss_func(y, y_predict):
#     error = 0
#     for i in range(0, y.shape[1]):
#         error += mean_squared_error(y[:, i], y_predict[:, i])
#     error = error / y.shape[1]
#     return error

# my_scorer = make_scorer(my_custom_loss_func, greater_is_better=False)

#循环筛选变量
while variate:
    mse_with_variate_train = []
    mse_with_variate_val=[]
    for i in variate:  #逐个遍历自变量
        current_variate = selected+[i]   #当前变量
        current_x_train = x_train[:, current_variate]   #当前x_train
        current_x_test = x_test[:, current_variate]   #当前x_test
        
        train_mse_list_ = []
        val_mse_list_ = []
        
        for train_index, val_index in kf.split(current_x_train, y_train):
            x_train_ = current_x_train[train_index, :]
            x_val_ = current_x_train[val_index, :]
            y_train_ = y_train[train_index, :]
            y_val_ = y_train[val_index, :]
            
            #model = MLPRegressor(random_state=0,early_stopping=True,alpha=0.0001,hidden_layer_sizes=(1,),learning_rate_init=0.01,
            #                    max_iter=5000,n_iter_no_change=100)
            svr = SVR(kernel='rbf')
            multioutput_svr = MultiOutputRegressor(svr, n_jobs=-1)
            multioutput_svr.fit(x_train_, y_train_)
        
            train_pre = multioutput_svr.predict(x_train_)
            val_pre = multioutput_svr.predict(x_val_)
            
            train_mse_ = mean_squared_error(y_train_, train_pre)
    
            val_mse_ = mean_squared_error(y_val_, val_pre)
    
            train_mse_list_.append(train_mse_)
    
            val_mse_list_.append(val_mse_)
            
        train_mse = np.mean(train_mse_list_)
        
        val_mse = np.mean(val_mse_list_)
        
        mse_with_variate_train.append((train_mse,i))
        
        mse_with_variate_val.append((val_mse,i))
            
    min_mse_index = mse_with_variate_val.index(min(mse_with_variate_val))
    best_new_score,best_candidate_index = mse_with_variate_val[min_mse_index]
    train_new_score,_ = mse_with_variate_train[min_mse_index]
        
    variate.remove(best_candidate_index)  #移除加进来的变量名，即第二次循环时，不考虑此自变量了
    selected.append(best_candidate_index)  #将此自变量作为加进模型中的自变量
    
    train_mse_list.append(mse_with_variate_train[min_mse_index][0])
    
    val_mse_list.append(mse_with_variate_val[min_mse_index][0])
        
        #if current_score>best_new_score:
            #variate.remove(best_candidate_index)  #移除加进来的变量名，即第二次循环时，不考虑此自变量了
            #selected.append(best_candidate_index)  #将此自变量作为加进模型中的自变量
            #current_score = best_new_score  #最新的分数等于最好的分数
            #print("score is {},continuing!".format(current_score))  #输出最小的mse值
        #else:
            #print("for selection over!")
            #break

In [9]:
selected

[179,
 158,
 201,
 173,
 243,
 232,
 84,
 107,
 191,
 253,
 304,
 94,
 22,
 69,
 241,
 190,
 219,
 147,
 256,
 246,
 23,
 58,
 169,
 142,
 93,
 242,
 57,
 251,
 50,
 27,
 237,
 249,
 68,
 51,
 73,
 234,
 14,
 67,
 0,
 83,
 126,
 148,
 79,
 225,
 198,
 269,
 153,
 240,
 295,
 104,
 177,
 102,
 45,
 59,
 1,
 250,
 160,
 61,
 2,
 4,
 6,
 180,
 9,
 216,
 11,
 13,
 15,
 16,
 18,
 20,
 196,
 26,
 33,
 35,
 36,
 37,
 39,
 254,
 40,
 41,
 43,
 183,
 47,
 53,
 56,
 17,
 280,
 268,
 159,
 101,
 46,
 303,
 25,
 136,
 60,
 62,
 110,
 222,
 163,
 98,
 90,
 52,
 118,
 233,
 167,
 96,
 100,
 19,
 181,
 302,
 168,
 217,
 21,
 187,
 231,
 276,
 7,
 152,
 224,
 288,
 182,
 298,
 218,
 278,
 128,
 228,
 139,
 113,
 184,
 42,
 283,
 123,
 30,
 134,
 193,
 85,
 150,
 131,
 87,
 77,
 279,
 195,
 165,
 141,
 161,
 174,
 70,
 300,
 10,
 44,
 80,
 5,
 192,
 55,
 81,
 72,
 76,
 252,
 301,
 282,
 255,
 223,
 105,
 108,
 78,
 144,
 91,
 197,
 95,
 207,
 130,
 121,
 162,
 89,
 29,
 146,
 189,
 291,
 66,
 145,
 124

In [10]:
train_mse_list

[0.03689112851872299,
 0.03193566396627815,
 0.030092912758958555,
 0.02842007981180273,
 0.027335319539312185,
 0.024042904610180247,
 0.020473218117096105,
 0.018823224658866926,
 0.01816893446275944,
 0.017943684323057103,
 0.018032680221433454,
 0.0180348495926894,
 0.018293502601924562,
 0.018470722100608506,
 0.01850905100839009,
 0.01875790881035501,
 0.018733656208418818,
 0.018630547854127703,
 0.018641677108429473,
 0.0187104961634974,
 0.01883995280013539,
 0.018978138032582713,
 0.01878903974965903,
 0.017789582275733788,
 0.017687419558946156,
 0.017785354827384713,
 0.017796831346706097,
 0.017841215405890086,
 0.01680592320704491,
 0.01664201055312882,
 0.01676000302526838,
 0.016833804530330516,
 0.016800051115275803,
 0.01686057118371062,
 0.016670018035798442,
 0.01672417370473447,
 0.0167956005104195,
 0.01680710541333557,
 0.0168583685784329,
 0.016384598809416256,
 0.016407119576775066,
 0.016401684553353873,
 0.016296743485073817,
 0.016168069409719696,
 0.0162516

In [11]:
val_mse_list

[0.04124965370870033,
 0.03800751626610581,
 0.03696543982422525,
 0.03541704267700206,
 0.03570171765655056,
 0.03539912158158838,
 0.03381146770650131,
 0.03265711949028398,
 0.03077069532916188,
 0.029564209373383794,
 0.02930957171272385,
 0.029010381166621253,
 0.028968975157028315,
 0.028798711208822565,
 0.028841083787247833,
 0.028473651417180633,
 0.02846304149715937,
 0.02835916762871704,
 0.027938208719043532,
 0.027678763783207976,
 0.027400129785892895,
 0.027240925349243394,
 0.02703440202206146,
 0.02687090091958741,
 0.026564607776159634,
 0.02632241480591891,
 0.026117467538666457,
 0.02600098666245713,
 0.025915986270864482,
 0.0257003676418176,
 0.025659060371546992,
 0.025677020935623802,
 0.025654782191719044,
 0.025648550496601547,
 0.025710955356380627,
 0.025772080001879593,
 0.025875857024593628,
 0.025944054601258683,
 0.02601699082087952,
 0.026051056563008317,
 0.02591708661381247,
 0.02587700850288215,
 0.025750488199994336,
 0.025750445198573335,
 0.025753

In [12]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

plt.scatter(range(len(train_mse_list)), train_mse_list)
plt.show()

In [13]:
plt.scatter(range(len(val_mse_list)), val_mse_list)
plt.show()

In [7]:

min_index  = val_mse_list.index(min(val_mse_list))
min_value = val_mse_list[min_index]
print(min_index)
print(min_value)

91
0.02516433599562908


In [9]:
selected_columns = x_data.columns[selected[:min_index+1]].tolist()
for col in selected_columns:
    print(col)

Conju-Strut Wiener Index (Max-Conju)
NumAromaticRings
Fun_PEOE-Charge_Mean
Full-Mol Wiener Index
Conju_PEOE-Charge_Max
Mid-Fun_LogP_Delta
Chi2v
PEOE_VSA9
Atom_MR_Max
Occupied Orbitals
Atom1_ALIE Min value
PEOE_VSA1
Alcohol
ExactMolWt
Conju_LogP_Std
Atom_LogP_Max
Mid-Fun_PEOE-Charge_Mean
VSA_EState8
LUMO
Conju_PEOE-Charge_Min
Althoxyl
Sulfon
Num of Conju-Strut
VSA_EState3
LabuteASA
Conju_MR_Std
Nitro
Conju_MR_Delta
Arylchloride
Primary_arom_amine
Conju_PEOE-Charge_Mean
Conju_PEOE-Charge_Delta
HeavyAtomMolWt
Arylfluoride
MaxAbsPartialCharge
Conju_PEOE-Charge_Sum
c1ccc2sc_n+_c2c1
MolWt
c1nc_nH_n1
Chi2n
SlogP_VSA7
VSA_EState9
Chi0v
Mid-Fun_PEOE-Charge_Max
Fun_PEOE-Charge_Sum
Octopole Moment
NumAliphaticCarbocycles
Conju_PEOE-Charge_Std
ALIE Average Value
PEOE_VSA6
Max Conju-Distance (Max-Conju)
PEOE_VSA4
Amide
Sulfonic_derivative
c1c_nH_cn1
Conju_LogP_Delta
NumHDonors
Tri_boron
c1cc_nH_c1
c1ccc2_nH_ccc2c1
c1ccc2cc3ccccc3cc2c1
Atom_PEOE-Charge_Sum
c1cn_nH_c1
Mid-Fun_PEOE-Charge_Sum
c1cc2ccc

In [16]:
#col_index = [sorted_index[i] for i in selected[:min_index+1]]
# 指定图像保存的路径
import os
save_dir = 'Selected features'
os.makedirs(save_dir, exist_ok=True)  # 创建目录，如果目录不存在
data = x_data[selected_columns]
# 将选定的数据保存为 CSV 文件
output_file = os.path.join(save_dir, "selected_features-CO.csv")
data.to_csv(output_file, index=False, encoding='utf_8_sig')
data

Conju-Strut Wiener Index (Max-Conju)  NumAromaticRings  \
0                               1.023810                 1   
1                               0.866667                 1   
2                               0.666667                 0   
3                               0.666667                 0   
4                               0.500000                 0   
..                                   ...               ...   
77                              0.000000                 0   
78                              0.000000                 0   
79                              0.000000                 0   
80                              0.000000                 0   
81                              0.000000                 0   

    Fun_PEOE-Charge_Mean  Full-Mol Wiener Index  Conju_PEOE-Charge_Max  \
0           1.241938e-01               1.427273          -1.259678e-01   
1          -2.780000e-17               0.866667          -4.160000e-17   
2          -9.395815e-02               0.900000          -1.231819e-01   
3          -1.515180e-01               2.030303          -1.337200e-01   
4          -1.928926e-02               1.333333          -1.928926e-02   
..                   ...                    ...                    ...   
77         -7.716095e-02               0.000000           0.000000e+00   
78         -7.716095e-02               0.000000           0.000000e+00   
79         -7.716095e-02               0.000000           0.000000e+00   
80         -7.716043e-02               0.000000           0.000000e+00   
81         -7.627934e-02               0.000000           0.000000e+00   

    Mid-Fun_LogP_Delta     Chi2v  PEOE_VSA9  Atom_MR_Max  Occupied Orbitals  \
0               0.3808  3.247225   4.877147        6.691                 44   
1               0.0000  3.244094   3.786415        8.927                 39   
2               0.8027  0.868080   5.752854        7.591                 24   
3               1.7186  4.633542  12.841643        7.591                 55   
4               0.0000  2.189723   0.000000        7.591                 31   
..                 ...       ...        ...          ...                ...   
77              0.0000  4.170068   0.000000        7.591                 57   
78              0.0000  3.462961   0.000000        7.591                 49   
79              0.0000  2.402301   0.000000        7.591                 37   
80              0.0000  2.048748   0.000000        7.591                 33   
81              0.0000  1.341641   0.000000        7.591                 25   

    Atom1_ALIE Min value  PEOE_VSA1  Alcohol  ExactMolWt  Conju_LogP_Std  \
0               9.858382  10.048045        0  170.020895             0.0   
1               9.734129   0.000000        0  161.913883             0.0   
2               8.768344   5.106527        0   91.993200             0.0   
3               8.509234  10.213055        0  210.002051             0.0   
4               7.853093   0.000000        0  114.050321             0.0   
..                   ...        ...      ...         ...             ...   
77              8.026730   0.000000        0  206.116293             0.0   
78              8.027367   0.000000        0  178.084993             0.0   
79              8.035676   0.000000        0  136.038042             0.0   
80              8.049127   0.000000        0  122.022392             0.0   
81              8.158677   0.000000        0   93.991092             0.0   

    Atom_LogP_Max  Mid-Fun_PEOE-Charge_Mean  VSA_EState8      LUMO  \
0          0.6237              2.478850e-01     1.443818 -0.081651   
1          0.8456             -4.160000e-17     0.000000 -0.029607   
2          0.6482             -9.395815e-02     0.000000 -0.012591   
3          0.6482             -1.515180e-01     0.000000 -0.035858   
4          0.6482             -1.928926e-02     7.154722 -0.015570   
..            ...                       ...          ...       ...   
77         0.6482 

In [12]:

from sklearn.metrics import mean_squared_error, r2_score
# 用选出来的最佳变量在测试集上进行评估
# 使用min_index+1位置之前的selected变量
selected_columns = selected[:min_index+1]
final_x_train = x_train[:, selected_columns]
final_x_test = x_test[:, selected_columns]

svr = SVR(kernel='rbf')
multioutput_svr = MultiOutputRegressor(svr, n_jobs=-1)
multioutput_svr.fit(final_x_train, y_train)

test_pre = multioutput_svr.predict(final_x_test)
test_mse = mean_squared_error(y_test, test_pre)
test_r2 = r2_score(y_test, test_pre)

print("Selected variables:", selected_columns)
print("Test MSE:", test_mse)
print("Test R²:", test_r2)

# 创建保存目录
'''import os
save_dir = 'Test mse after svr'
os.makedirs(save_dir, exist_ok=True)  # 创建目录，如果目录不存在
output_file1 = os.path.join(save_dir, "final_evaluation-CO.csv")
# 保存测试集评估结果到CSV
df_summary = pd.DataFrame({
    'Selected Variables': [selected_columns],
    'Final Test MSE': [test_mse],
    'Final Test R2': [test_r2]
})
df_summary.to_csv(output_file1, index=False)

print(f"Results saved to '{output_file1}'")'''

Selected variables: [179, 158, 201, 173, 243, 232, 84, 107, 191, 253, 304, 94, 22, 69, 241, 190, 219, 147, 256, 246, 23, 58, 169, 142, 93, 242, 57, 251, 50, 27, 237, 249, 68, 51, 73, 234, 14, 67, 0, 83, 126, 148, 79, 225, 198, 269, 153, 240, 295, 104, 177, 102, 45, 59, 1, 250, 160, 61, 2, 4, 6, 180, 9, 216, 11, 13, 15, 16, 18, 20, 196, 26, 33, 35, 36, 37, 39, 254, 40, 41, 43, 183, 47, 53, 56, 17, 280, 268, 159, 101, 46, 303]
Test MSE: 0.06750381378674469
Test R²: 0.4449112145134445


'import os\nsave_dir = \'Test mse after svr\'\nos.makedirs(save_dir, exist_ok=True)  # 创建目录，如果目录不存在\noutput_file1 = os.path.join(save_dir, "final_evaluation-CO.csv")\n# 保存测试集评估结果到CSV\ndf_summary = pd.DataFrame({\n    \'Selected Variables\': [selected_columns],\n    \'Final Test MSE\': [test_mse],\n    \'Final Test R2\': [test_r2]\n})\ndf_summary.to_csv(output_file1, index=False)\n\nprint(f"Results saved to \'{output_file1}\'")'